In [ ]:
#connect

import sqlite3
import pandas as pd
import mariadb
import secret


def main():
    conn = mariadb.connect(user=secret.db_username,
                           password=secret.db_password,
                           host=secret.db_host,
                           port=secret.db_port,
                           database=secret.db_databasename)
    cur = conn.cursor()

## 스키마 변경 전(sqlite schema 그대로)

In [ ]:
cur.execute(
    '''CREATE TABLE IF NOT EXISTS BlkID(
       id INT NOT NULL,
       blkhash CHAR(64) NOT NULL,
       PRIMARY KEY(id),
       UNIQUE(blkhash));''')
cur.execute(
    '''CREATE TABLE IF NOT EXISTS TxID(
       id INT NOT NULL,
       txid CHAR(64) NOT NULL,
       PRIMARY KEY(id),
       UNIQUE(txid));''')
cur.execute(
    '''CREATE TABLE IF NOT EXISTS AddrID(
       id INT NOT NULL,
       addr CHAR(64) NOT NULL,
       PRIMARY KEY(id),
       UNIQUE(addr));''')
cur.execute(
    '''CREATE TABLE IF NOT EXISTS BlkTx(
       blk INT NOT NULL,
       tx INT NOT NULL,
       UNIQUE(blk, tx));''')
cur.execute(
    '''CREATE TABLE IF NOT EXISTS BlkTime(
       blk INT NOT NULL,
       unixtime INT NOT NULL,
       PRIMARY KEY(blk));''')
cur.execute(
    '''CREATE TABLE IF NOT EXISTS TxIn(
       tx INT NOT NULL,
       n INT NOT NULL,
       ptx INT NOT NULL,
       pn INT NOT NULL,
       UNIQUE(tx, n));''')
cur.execute(
    '''CREATE TABLE IF NOT EXISTS TxOut(
       tx INT NOT NULL,
       n INT NOT NULL,
       addr INT NOT NULL,
       btc DOUBLE NOT NULL,
       UNIQUE(tx, n, addr));''')

conn.commit()

load csv file to mariaDB

In [ ]:
#BlkID
cur.execute('SET autocommit = 0;')
cur.execute('SET unique_checks = 0;')
cur.execute('load data local infile "BlkID.csv" into table BlkID columns terminated by "," enclosed by "\n" ignore 1 rows;')
conn.commit()
cur.execute('SET autocommit = 1;')
cur.execute('SET unique_checks = 1;')

In [ ]:
#TxID
cur.execute('SET autocommit = 0;')
cur.execute('SET unique_checks = 0;')
cur.execute('load data local infile "TxID.csv" into table TxID columns terminated by "," enclosed by "\n" ignore 1 rows;')
conn.commit()
cur.execute('SET autocommit = 1;')
cur.execute('SET unique_checks = 1;')

In [ ]:
#AddrID
cur.execute('SET autocommit = 0;')
cur.execute('SET unique_checks = 0;')
cur.execute('load data local infile "AddrID.csv" into table AddrID columns terminated by "," lines terminated by "\r\n" ignore 1 rows;')
conn.commit()
cur.execute('SET autocommit = 1;')
cur.execute('SET unique_checks = 1;')

In [ ]:
#BlkTx
cur.execute('SET autocommit = 0;')
cur.execute('SET unique_checks = 0;')
cur.execute('load data local infile "BlkTx.csv" into table BlkTx columns terminated by "," enclosed by "\n" ignore 1 rows;')
conn.commit()
cur.execute('SET autocommit = 1;')
cur.execute('SET unique_checks = 1;')

In [ ]:
#BlkTime
cur.execute('SET autocommit = 0;')
cur.execute('SET unique_checks = 0;')
cur.execute('load data local infile "BlkTime.csv" into table BlkTime columns terminated by "," enclosed by "\n" ignore 1 rows;')
conn.commit()
cur.execute('SET autocommit = 1;')
cur.execute('SET unique_checks = 1;')

In [ ]:
#TxIn
cur.execute('SET autocommit = 0;')
cur.execute('SET unique_checks = 0;')
cur.execute('load data local infile "TxIn.csv" into table TxIn columns terminated by "," enclosed by "\n" ignore 1 rows;')
conn.commit()
cur.execute('SET autocommit = 1;')
cur.execute('SET unique_checks = 1;')

In [ ]:
#TxOut
cur.execute('SET autocommit = 0;')
cur.execute('SET unique_checks = 0;')
cur.execute('load data local infile "TxOut.csv" into table TxOut columns terminated by "," enclosed by "\n" ignore 1 rows;')
conn.commit()
cur.execute('SET autocommit = 1;')
cur.execute('SET unique_checks = 1;')

## 스키마 변경 후

In [1]:
cur.execute(
    '''CREATE TABLE IF NOT EXISTS blkinfo(
       blkid INT NOT NULL,
       blkhash CHAR(64) NOT NULL,
       miningtime INT NOT NULL,
       PRIMARY KEY(blkid),
       UNIQUE(blkhash, miningtime));''')

cur.execute(
    '''CREATE TABLE IF NOT EXISTS txinfo(
       txid INT NOT NULL,
       txhash CHAR(64) NOT NULL,
       blkid INT NOT NULL,
       PRIMARY KEY(txid),
       FOREIGN KEY(blkid) REFERENCES blkinfo(blkid) ON DELETE CASCADE,
       UNIQUE(txhash, blkid));''')

cur.execute(
    '''CREATE TABLE IF NOT EXISTS addrtype(
       addrtypeid INT NOT NULL,
       addrtype TEXT NOT NULL,
       PRIMARY KEY(addrtypeid),
       UNIQUE(addrtype));''')

cur.execute(
    '''CREATE TABLE IF NOT EXISTS addrinfo(
       addrid INT NOT NULL,
       addr TEXT NOT NULL,
       addrtypeid INT NOT NULL,
       blkid INT NOT NULL,
       PRIMARY KEY(addrid),
       FOREIGN KEY(blkid) REFERENCES blkinfo(blkid) ON DELETE CASCADE,
       FOREIGN KEY(addrtypeid) REFERENCES addrtype(addrtypeid),
       UNIQUE(addr, addrtypeid, blkid));''')

cur.execute(
    '''CREATE TABLE IF NOT EXISTS txin(
       txid INT NOT NULL,
       n INT NOT NULL,
       ptxid INT NOT NULL,
       pn INT NOT NULL,
       FOREIGN KEY(txid) REFERENCES txinfo(txid) ON DELETE CASCADE,
       UNIQUE(txid, n));''')

cur.execute(
    '''CREATE TABLE IF NOT EXISTS txout(
       txid INT NOT NULL,
       n INT NOT NULL,
       addrid INT NOT NULL,
       btc DOUBLE NOT NULL,
       FOREIGN KEY(txid) REFERENCES txinfo(txid) ON DELETE CASCADE,
       UNIQUE(txid, n));''')

conn.commit()

load csv file to mariaDB

In [ ]:
cur.execute('load data local infile "blkinfo.csv" into table blkinfo columns terminated by "," enclosed by "\n" ignore 1 rows;')

cur.execute('load data local infile "txinfo.csv" into table txinfo columns terminated by "," enclosed by "\n" ignore 1 rows;')

cur.execute('load data local infile "addrtype.csv" into table addrtype columns terminated by "," enclosed by "\n" ignore 1 rows;')

cur.execute('load data local infile "addrinfo.csv" into table addrinfo columns terminated by "," enclosed by "\n" ignore 1 rows;')

cur.execute('load data local infile "txin.csv" into table txin columns terminated by "," enclosed by "\n" ignore 1 rows;')

cur.execute('load data local infile "txout.csv" into table txout columns terminated by "," enclosed by "\n" ignore 1 rows;')

conn.commit()